python 魔术方法是什么？
A：在python中所有双下滑线开始和结尾的方法。都叫魔术方法。

Q: 调用len()的是时候发生了什么？
A：调用`len(str)`的时候其实调用的`str.__len__()`方法。

## 线性解构

特点： 1.内存连续 3. 下标访问

In [1]:
python 有内置类型 array  只能存放同一类型。数字和字符。

from array import array

list 的工作原理：及操作。看cpython实现。
创建 O（1）
append 时候新分配内存空间。如果第一次分配空间不够，会重新分配内存空间。 第一次4个， 第二次0～8 resize  16  25 



insert O（n）

remove （n） 频繁的进行insert 和remove就成（）



Q：实现一个定长的数组：


__iter__()方法，用来遍历。
__setitem__() 方法。
__getitem__() 方法。
首先，实现不变容器的话有一个协议:实现不可变容器，你只能定义 __len__ 和 __getitem__ (一会会讲更多)。
可变容器协议则需要所有不可变容器的所有另外还需要 __setitem__ 和 __delitem__ 。
最终，如果你希望你的对象是可迭代的话，你需要定义 __iter__ 会返回一个迭代器。
迭代器必须遵循迭代器协议，需要有 __iter__ (返回它本身) 和 next 。

In [6]:
class Array():
    def __init__(self, size=32):
        self._size = size
        self._items = [None] * size
    def __getitem__(self, index):   
        return self._items[index]
    def __setitem__(self, index, value):
        self._items[index] = value
    def __len__(self):
        return self._size
    def clean(self, value=None):
        for i in range(len(self._items)):
            self._items[i] = value
            
    def __iter__(self):
        for item in self._items:
            yield item
            
def test_array():
    size = 10 
    a = Array(size)
    a[0] = 1
    a.clearn()
    assert a[0] is None

In [7]:
x=Array(4)


In [8]:
len(x)

4

In [15]:
x[0]=4
print(x)

In [16]:
print(x[0])

4


In [9]:
print(x.__len__())

None


***

## 单链表


线性表： 内存连续，下标访问
链表： 不连续，无[], append 

In [24]:
# -*- coding: utf-8 -*-


class Node(object):
    def __init__(self, value=None, next=None):   # 这里我们 root 节点默认都是 None，所以都给了默认值
        self.value = value
        self.next = next

    def __str__(self):
        """方便你打出来调试，复杂的代码可能需要断点调试"""
        return '<Node: value: {}, next={}>'.format(self.value, self.next)

    __repr__ = __str__


class LinkedList(object):
    """ 链接表 ADT
    [root] -> [node0] -> [node1] -> [node2]
    """

    def __init__(self, maxsize=None):
        """
        :param maxsize: int or None, 如果是 None，无限扩充
        """
        self.maxsize = maxsize
        self.root = Node()     # 默认 root 节点指向 None
        self.tailnode = None
        self.length = 0

    def __len__(self):
        return self.length

    def append(self, value):    # O(1)
        if self.maxsize is not None and len(self) >= self.maxsize:
            raise Exception('LinkedList is Full')
        node = Node(value)    # 构造节点
        tailnode = self.tailnode
        if tailnode is None:    # 还没有 append 过，length = 0， 追加到 root 后
            self.root.next = node
        else:     # 否则追加到最后一个节点的后边，并更新最后一个节点是 append 的节点
            tailnode.next = node
        self.tailnode = node
        self.length += 1

    def appendleft(self, value):
        if self.maxsize is not None and len(self) >= self.maxsize:
            raise Exception('LinkedList is Full')
        node = Node(value)
        if self.tailnode is None:  # 如果原链表为空，插入第一个元素需要设置 tailnode
            self.tailnode = node

        headnode = self.root.next
        self.root.next = node
        node.next = headnode
        self.length += 1

    def __iter__(self):
        for node in self.iter_node():
            yield node.value

    def iter_node(self):
        """遍历 从 head 节点到 tail 节点"""
        curnode = self.root.next
        while curnode is not self.tailnode:    # 从第一个节点开始遍历
            yield curnode
            curnode = curnode.next    # 移动到下一个节点
        if curnode is not None:
            yield curnode

    def remove(self, value):    # O(n)
        """ 删除包含值的一个节点，将其前一个节点的 next 指向被查询节点的下一个即可
        :param value:
        """
        prevnode = self.root    #
        for curnode in self.iter_node():
            if curnode.value == value:
                prevnode.next = curnode.next
                if curnode is self.tailnode:  # NOTE: 注意更新 tailnode
                    if prevnode is self.root:
                        self.tailnode = None
                    else:
                        self.tailnode = prevnode
                del curnode
                self.length -= 1
                return 1  # 表明删除成功
            else:
                prevnode = curnode
        return -1  # 表明删除失败

    def find(self, value):    # O(n)
        """ 查找一个节点，返回序号，从 0 开始
        :param value:
        """
        index = 0
        for node in self.iter_node():   # 我们定义了 __iter__，这里就可以用 for 遍历它了
            if node.value == value:
                return index
            index += 1
        return -1    # 没找到

    def popleft(self):    # O(1)
        """ 删除第一个链表节点
        """
        if self.root.next is None:
            raise Exception('pop from empty LinkedList')
        headnode = self.root.next
        self.root.next = headnode.next
        self.length -= 1
        value = headnode.value

        if self.tailnode is headnode:   # 勘误：增加单节点删除 tailnode 处理
            self.tailnode = None
        del headnode
        return value

    def clear(self):
        for node in self.iter_node():
            del node
        self.root.next = None
        self.length = 0
        self.tailnode = None

    def reverse(self):
        """反转链表"""
        curnode = self.root.next
        self.tailnode = curnode  # 记得更新 tailnode，多了这个属性处理起来经常忘记
        prevnode = None

        while curnode:
            nextnode = curnode.next
            curnode.next = prevnode

            if nextnode is None:
                self.root.next = curnode

            prevnode = curnode
            curnode = nextnode


def test_linked_list():
    ll = LinkedList()

    ll.append(0)
    ll.append(1)
    ll.append(2)
    ll.append(3)

    assert len(ll) == 4
    assert ll.find(2) == 2
    assert ll.find(-1) == -1

    assert ll.remove(0) == 1
    assert ll.remove(10) == -1
    assert ll.remove(2) == 1
    assert len(ll) == 2
    assert list(ll) == [1, 3]
    assert ll.find(0) == -1

    ll.appendleft(0)
    assert list(ll) == [0, 1, 3]
    assert len(ll) == 3

    headvalue = ll.popleft()
    assert headvalue == 0
    assert len(ll) == 2
    assert list(ll) == [1, 3]

    assert ll.popleft() == 1
    assert list(ll) == [3]
    ll.popleft()
    assert len(ll) == 0
    assert ll.tailnode is None

    ll.clear()
    assert len(ll) == 0
    assert list(ll) == []


def test_linked_list_remove():
    ll = LinkedList()
    ll.append(3)
    ll.append(4)
    ll.append(5)
    ll.append(6)
    ll.append(7)
    ll.remove(7)
    print(list(ll))

def test_single_node():
    # https://github.com/PegasusWang/python_data_structures_and_algorithms/pull/21
    ll = LinkedList()
    ll.append(0)
    ll.remove(0)
    ll.appendleft(1)
    assert list(ll) == [1]

def test_linked_list_reverse():
    ll = LinkedList()
    n = 10
    for i in range(n):
        ll.append(i)
    ll.reverse()
    assert list(ll) == list(reversed(range(n)))


def test_linked_list_append():
    ll = LinkedList()
    ll.appendleft(1)
    ll.append(2)
    assert list(ll) == [1, 2]


if __name__ == '__main__':
    test_single_node()
    test_linked_list()
    test_linked_list_remove()
    test_linked_list_append()
    test_linked_list_reverse()

[3, 4, 5, 6]


***
## 循环双链表

In [37]:
class  Node(object):
    __slots__ = ('value', 'prev', 'next')
    def __init__(self ,value=None, prev=None, next=None):
        self.value, self.prev, self.next = value, prev, next
 

提前定义几个概念：根节点，rootnode 头节点，headnode 头节点是根的下一个。 尾节点。tailnode

appleft是插入到rootnode的下一个节点。也就是在原始root和head中间插入值。

In [48]:


class CircularDoubleLinkedList(object):
    
    def __init__(self, maxsize=None):
        self.maxsize = maxsize
        node = Node()
        node.next, node.prev = node, node
        self.root = node 
        self.length = 0 
    
    def __len__(self):
        return self.length
    
    def headnode(self):
        return self.root.next
    
    def tailnode(self):
        return self.root.prev
    
    def append(self, value): 
        if self.maxsize is not None and len(self) >= self.maxsize:
            raise Exption('LinkedList is Full')
        node = Node(value=value)
        tailnode = self.tailnode() or self.root
        
        tailnode.next = node   #赋值改4次。
        node.prev = tailnode 
        node.next = self.root
        self.root.prev = node 
        self.length +=1
        
    def appendleft(self, value):
        if self.maxsize is not None and len(self) >=self.maxsize:
            raise Exception('LinkedList is Full')
        node = Node(value=value)
        if self.root.next is self.root:
            node.next = self.root
            node.prev = self.root
            self.root.next =node 
            self.root.prev = node
        else:
            node.prev= self.root
            headnode = self.root.next
            node.next = headnode
            headnode.prev = node
            self.root.next = node 
        self.length +=1 
                
    def remove(self,node):
        if node is self.root:
            return
        else:
            node.prev.next = node.next
            node.next.prev = node.prev
        self.length -=1 
        return node 
    
    def iter_node(self):
        if self.root.next is self.root:
            return 
        curnode= self.root.next
        while curnode.next is not self.root:
            yield curnode 
            curnode = curnode.next
        yield curnode 
    
    def __iter__(self):
        for node in self.iter_node():
            yield node.value
    def iter_node_reverse(self):
        if self.root.prev is self.root:
            return 
        curnode = self.root.prev
        while curnode.prev is not self.root:
            yield curnode 
            curnode = curnode.prev
        yield curnode
        
        
def test_double_link_list():
    dll = CircularDoubleLinkedList()
    assert len(dll) == 0
    
    dll.append(0)
    dll.append(1)
    dll.append(2)
    
    assert list(dll) == [0,1,2]
    
    assert [node.value for node in dll.iter_node()] == [0,1,2]
    assert [node.value for node in dll.iter_node_reverse()] ==[2,1,0]

    headnode = dll.headnode()
    assert headnode.value == 0
    dll.remove(headnode)
    assert len(dll) == 2
    assert [node.value for node in dll.iter_node()] == [1,2]
    
    dll.appendleft(0)
    
    assert [node.value for node in dll.iter_node()] == [0,1,2]
    print('ok')
if __name__ =='__main__':
    test_double_link_list()
        
        

ok


In [49]:
# 双段队列
def Deque(CircularDoubleLinkedList):
    def pop(self):
        if len(self) == 0:
            return Exception('empty')
        tailnode =self.headnode()
        value = headnode.value
        self.remove(headnode)
    
    def 

    
# collections.deque



SyntaxError: invalid syntax (<ipython-input-49-237a3568ca64>, line 10)

In [50]:
## 哈希冲突解决方法
取模找槽：然后把值放入余数位置。 然后多处理来的数字放入余数槽后面。





SyntaxError: invalid character in identifier (<ipython-input-50-fa7cde4a42b2>, line 2)

In [53]:
#集合 交并集要运算符重置

# 
def hanoi_move(n, source, dest, intermediate):
    if n >= 1:  # 递归出口，只剩一个盘子
        hanoi_move(n-1, source, intermediate, dest)
        print("Move %s -> %s" % (source, dest))
        hanoi_move(n-1, intermediate, dest, source)
hanoi_move(4, '1', '2', '3')

Move 1 -> 3
Move 1 -> 2
Move 3 -> 2
Move 1 -> 3
Move 2 -> 1
Move 2 -> 3
Move 1 -> 3
Move 1 -> 2
Move 3 -> 2
Move 3 -> 1
Move 2 -> 1
Move 3 -> 2
Move 1 -> 3
Move 1 -> 2
Move 3 -> 2
